Importing Libraries

In [1]:
import pandas as pd
import numpy as np

Reading the dataset

In [2]:
df = pd.read_csv(r"C:\Users\lokit\Downloads\uber_rides_data.xlsx - sample_train.csv")

In [3]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [4]:
#Q!What is the shape of given dataset?

In [5]:
df.shape

(200000, 8)

In [6]:
#Q2How many integer columns(by default) are given in the dataset?


In [7]:
df.dtypes

ride_id                int64
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [8]:
df.select_dtypes(include='int64')

,ride_id,passenger_count
0,24238194,1
1,27835199,1
2,44984355,1
3,25894730,3
4,17610152,5
...,...,...
199995,42598914,1
199996,16382965,1
199997,27804658,2
199998,20259894,1


In [9]:
#q3 How many missing values exists in 'dropoff_longitude' column?


In [10]:
df.columns

Index(['ride_id', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [11]:
df['dropoff_longitude'].isnull().sum()

1

In [12]:
df.isnull().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64

In [13]:
#Q4 What is the data type of ' pickup_datetime' feature in your data?


In [14]:
df['pickup_datetime'].dtypes

dtype('O')

In [15]:
df.dtypes

ride_id                int64
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [16]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [17]:
df.dtypes

ride_id                            int64
fare_amount                      float64
pickup_datetime      datetime64[ns, UTC]
pickup_longitude                 float64
pickup_latitude                  float64
dropoff_longitude                float64
dropoff_latitude                 float64
passenger_count                    int64
dtype: object

In [18]:
#Q5Which function can be used to remove null values from the dataframe?


In [19]:
df.dropna(inplace=True)

In [20]:
#Q6What is the average fare amount?

#Remove the null values from the dataframe to answer the following question.

In [21]:
np.mean(df['fare_amount'])

11.359891549458371

In [22]:
#q7 Calculate distance between each pickup and dropoff points using Haversine formula. 
#What is the median haversine distance between pickup and dropoff location according to the given dataset?

In [23]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    R = 6371.0 # Earth's radius in km
    dist = R * c
    
    return dist

In [24]:
df['distance'] = df.apply(lambda row: haversine_distance(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [25]:
df['distance']

0          1.683323
1          2.457590
2          5.036377
3          1.661683
4          4.475450
            ...    
199995     0.112210
199996     1.875050
199997    12.850319
199998     3.539715
199999     5.417783
Name: distance, Length: 199999, dtype: float64

In [26]:
np.median(df['distance'])

2.1209923961833708

In [27]:
#Q8 What is the maximum haversine distance between pickup and dropoff location according to the given dataset?

In [28]:
max(df['distance'])

16409.239135313168

In [29]:
# Q9 How many rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset?

In [30]:
len(df.loc[df['distance']==0])

5632

In [31]:
#Q What is the mean 'fare_amount' for rides with 0 haversine distance?

#Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.

In [32]:
df.loc[df['distance']==0]['fare_amount'].mean()

11.585317826704578

In [33]:
df.groupby('distance')['fare_amount'].mean()

distance
0.000000        11.585318
0.000084        50.000000
0.000084        45.000000
0.000084         2.500000
0.000084         2.500000
                  ...    
8708.233063      6.100000
8776.106278     49.800000
8782.898606      4.100000
10321.507661    13.700000
16409.239135    15.700000
Name: fare_amount, Length: 194361, dtype: float64

In [34]:
#Opinion : Even though the distance travelled is ZERo, passengers have been  charged with some fare which averages to 11.59
#Which can be interpreted as cancellation fee.Maybe the passenger has booked a ride and the ride maybe cancelled by the passenger after
#the exempted time.Since the driver travelled soome distance to reach the passengers location some fare might bee charged.(for cancellation)

In [35]:
#Q What is the maximum 'fare_amount' for a ride?


In [36]:
max(df['fare_amount'])

499.0

In [37]:
#What is the haversine distance between pickup and dropoff location for the costliest ride?

#Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.

In [38]:
df.sort_values('fare_amount',ascending=False).head(1)

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
170081,51151143,499.0,2011-04-10 04:10:00+00:00,-73.968377,40.764602,-73.968368,40.7646,1,0.00079


In [39]:
#Opinion: The fare amount is maximum for a very short distance.It could be due timing(4:10) as we can observe,where no or very less
# cabs may be available due to which the price may go high(peak time).
# Bandh may be happening
#Any strike(Auto) may be going on at that time.
#Maybe it could be an area like airport where the cab prices are high even for a short distance.
#It can also be due to the piled up cancellation fee or the fare which passenger might not have paid for the earlier ride.
#past transaction might have failed due to which the amount might have added up for this ride.


In [40]:
#How many rides were recorded in the year 2014?


In [41]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450


In [42]:
import datetime as dt

In [43]:

df['pickup_year'] = df['pickup_datetime'].dt.year

In [44]:
len(df[df['pickup_year']==2014])

29968

In [45]:
#Q)How many rides were recorded in the first quarter of 2014?


In [46]:
def month_to_quarter (month):
    if month in [1,2,3]:
        return 'Q1'
    elif month in [4,5,6]:
        return 'Q2'
    elif month in [7,8,9]:
        return 'Q3'
    else: return 'Q4'

In [47]:
df['pickup_month'] = df['pickup_datetime'].dt.month

In [48]:
df['quater'] = df['pickup_month'].apply(month_to_quarter)

In [49]:
df['quater']

0         Q2
1         Q3
2         Q3
3         Q2
4         Q3
          ..
199995    Q4
199996    Q1
199997    Q2
199998    Q2
199999    Q2
Name: quater, Length: 199999, dtype: object

In [50]:
len(df.loc[(df['pickup_year'] ==2014) & (df['quater'] == 'Q1')])

7687

In [51]:
#On which day of the week in September 2010, maximum rides were recorded ?

In [52]:
df['day'] = df['pickup_datetime'].dt.strftime('%A')

In [53]:
df['ride_week_day'] = df['pickup_datetime'].dt.weekday

In [54]:
df['day']

0          Thursday
1            Friday
2            Monday
3            Friday
4          Thursday
            ...    
199995       Sunday
199996       Friday
199997       Monday
199998    Wednesday
199999     Saturday
Name: day, Length: 199999, dtype: object

In [55]:
df['ride_week_day']

0         3
1         4
2         0
3         4
4         3
         ..
199995    6
199996    4
199997    0
199998    2
199999    5
Name: ride_week_day, Length: 199999, dtype: int64

In [56]:
df['month'] = df['pickup_datetime'].dt.strftime('%B')

In [57]:
df1 = df.loc[(df['month']=='September') & (df['pickup_year']==2010)]

In [58]:
df1.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,pickup_year,pickup_month,quater,day,ride_week_day,month
23,25121708,7.7,2010-09-04 16:12:00+00:00,-73.994300,40.739512,-73.988070,40.724482,2,1.751763,2010,9,Q3,Saturday,5,September
85,11536406,7.7,2010-09-08 10:16:01+00:00,-73.987383,40.721255,-73.954733,40.730683,1,2.944304,2010,9,Q3,Wednesday,2,September
87,11267091,4.5,2010-09-22 12:15:00+00:00,-73.981920,40.761172,-74.016152,40.704765,1,6.903596,2010,9,Q3,Wednesday,2,September
126,8791172,5.3,2010-09-21 12:38:00+00:00,-73.956158,40.781593,-73.966138,40.773645,1,1.219522,2010,9,Q3,Tuesday,1,September
129,30203807,10.5,2010-09-10 16:28:00+00:00,-73.995627,40.749295,-73.965087,40.771610,1,3.573956,2010,9,Q3,Friday,4,September


In [59]:
df1['day'].value_counts()

Thursday     457
Wednesday    391
Saturday     362
Friday       354
Sunday       331
Tuesday      322
Monday       265
Name: day, dtype: int64

In [60]:
# Q
'''Apply a Machine Learning Algorithm to predict the fare amount given following input features:
passenger_count, distance and ride_week_day.

Perform a 70-30 split of data.

Which algorithm gives the least adjusted R square value?'''

'Apply a Machine Learning Algorithm to predict the fare amount given following input features:\npassenger_count, distance and ride_week_day.\n\nPerform a 70-30 split of data.\n\nWhich algorithm gives the least adjusted R square value?'

Importing algorithms

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

In [62]:
X= df[['passenger_count','distance','ride_week_day']]
y= df['fare_amount']

train test split

In [63]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [64]:
from sklearn.preprocessing import StandardScaler

Scaling the distance feature

In [65]:
scaler = StandardScaler()

In [66]:
x_train_s = scaler.fit_transform(x_train[['distance']])
x_train_d = pd.DataFrame(x_train_s,columns=['distance'])

In [67]:
x_test_s = scaler.transform(x_test[['distance']])
x_test_d = pd.DataFrame(x_test_s,columns=['distance'])

In [68]:
col = x_train[['passenger_count','ride_week_day']]
x_train1 = col.join(x_train_d,on=x_train_d.index)

In [69]:
col1 = x_test[['passenger_count','ride_week_day']]
x_test1 = col1.join(x_test_d,on=x_test_d.index)

Linear regression

In [70]:
lr = LinearRegression()

In [71]:
lr.fit(x_train1,y_train)

LinearRegression()

In [72]:
pred = lr.predict(x_test1)

In [73]:
lr.score(x_train1,y_train)

0.0005366865838487556

In [74]:
from sklearn.metrics import r2_score

In [75]:
r2_score(y_test,pred)

0.0011912025239576574

In [79]:
r2 = r2_score(y_test,pred)
n = x_test1.shape[0]
p = x_test1.shape[1]

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.0011412587544992192

Decision tree regressor

In [80]:
dt = DecisionTreeRegressor()

In [81]:
dt.fit(x_train1,y_train)

DecisionTreeRegressor()

In [82]:
pred_dt = dt.predict(x_test1)

In [92]:
r2_score(y_test,pred_dt)

0.48869787499500394

In [93]:
r2 = r2_score(y_test,pred_dt)
n = x_test1.shape[0]
p = x_test1.shape[1]

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.48867230818429963

Random forest Regressor

In [85]:
rf = RandomForestRegressor()

In [95]:
rf.fit(x_train1,y_train)

RandomForestRegressor()

In [96]:
pred_rf = rf.predict(x_test1)

In [97]:
r2_score(y_test,pred_rf)

0.6392985854125959

In [98]:
r2 = r2_score(y_test,pred_rf)
n = x_test1.shape[0]
p = x_test1.shape[1]

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.6392805491394482

Knn regressor

In [99]:
knn = KNeighborsRegressor()

In [100]:
knn.fit(x_train1,y_train)

KNeighborsRegressor()

In [101]:
pred_knn = knn.predict(x_test1)

In [102]:
r2 = r2_score(y_test,pred_knn)
n = x_test1.shape[0]
p = x_test1.shape[1]

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.6399288926412312